In [1]:
%matplotlib

import pandas as pd
import networkx as nx
from pyvis.network import Network
from jaal import Jaal
import matplotlib.pyplot as plt
import numpy as np
import itertools
from matplotlib import cm

Using matplotlib backend: Qt5Agg


C:\Users\mikejseay\anaconda3\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
# the idea here is to map genres to color and then color the style based on the genre it's most closely associate with
# since there are 15 genres, we will need 15 distinct colors
# the 'tab20' color map should be good for that
# but actually it doesn't matter because the jaal library has 15+ distinct colors

# that's it!

In [181]:
genreSizes = pd.read_csv('C:/Users/mikejseay/Documents/Code/discogs-xml2db/output_csv/genreFrequencies.csv')
genreSizes.set_index('genre', inplace=True)
genreSizes['id'] = range(genreSizes.shape[0])

genreWeights = pd.read_csv('C:/Users/mikejseay/Documents/Code/discogs-xml2db/output_csv/genreCoOccurences.csv')
genreWeights.rename(columns={'cnt': 'cooccurrence'}, inplace=True)

normFactor1 = []`
normFactor2 = []
for g1, g2, cooccurences in genreWeights[['g1', 'g2', 'cooccurrence']].values:
    normFactor1.append(np.sqrt(genreSizes.loc[g1, 'cnt'] * genreSizes.loc[g2, 'cnt']))  # Oichai similarity
    normFactor2.append(genreSizes.loc[g1, 'cnt'] + genreSizes.loc[g2, 'cnt'] - cooccurences)  # Jaccard similarity

genreWeights['normFactor1'] = normFactor1
genreWeights['normFactor2'] = normFactor2
genreWeights['weight1'] = genreWeights['cooccurrence'] / genreWeights['normFactor1']
genreWeights['weight2'] = genreWeights['cooccurrence'] / genreWeights['normFactor2']
genreWeights['weight'] = genreWeights['weight2']

label2id_genre = genreSizes['id'].to_dict()
genreWeights['from'] = genreWeights['g1'].replace(label2id_genre)
genreWeights['to'] = genreWeights['g2'].replace(label2id_genre)

In [182]:
genreStyleMapping = pd.read_csv('C:/Users/mikejseay/Documents/Code/discogs-xml2db/output_csv/genreStyleMapping.csv')
genreStyleMapping.set_index('style', inplace=True)

styleSizes = pd.read_csv('C:/Users/mikejseay/Documents/Code/discogs-xml2db/output_csv/styleFrequencies.csv').set_index('style').join(genreStyleMapping)
styleSizes.rename(columns={'cnt': 'cnt'}, inplace=True)
styleSizes['id'] = range(styleSizes.shape[0])

styleWeights = pd.read_csv('C:/Users/mikejseay/Documents/Code/discogs-xml2db/output_csv/styleCoOccurences.csv')
styleWeights.rename(columns={'cnt': 'cooccurrence'}, inplace=True)

normFactor1 = []
normFactor2 = []
for g1, g2, cooccurences in styleWeights[['g1', 'g2', 'cooccurrence']].values:
    normFactor1.append(np.sqrt(styleSizes.loc[g1, 'cnt'] * styleSizes.loc[g2, 'cnt']))  # Oichai similarity
    normFactor2.append(styleSizes.loc[g1, 'cnt'] + styleSizes.loc[g2, 'cnt'] - cooccurences)  # Jaccard similarity

styleWeights['normFactor1'] = normFactor1
styleWeights['normFactor2'] = normFactor2
styleWeights['weight1'] = styleWeights['cooccurrence'] / styleWeights['normFactor1']
styleWeights['weight2'] = styleWeights['cooccurrence'] / styleWeights['normFactor2']
styleWeights['weight'] = styleWeights['weight2']

label2id_style = styleSizes['id'].to_dict()
styleWeights['from'] = styleWeights['g1'].replace(label2id_style)
styleWeights['to'] = styleWeights['g2'].replace(label2id_style)

In [56]:
genreWeights.shape[0] / (genreSizes.shape[0] * (genreSizes.shape[0] - 1) / 2)

1.0

In [64]:
styleWeights.shape[0]

45794

In [57]:
styleWeights.shape[0] / (styleSizes.shape[0] * (styleSizes.shape[0] - 1) / 2)

0.2531439848314824

In [33]:
f, ax = plt.subplots()
ax.scatter(styleWeights['weight1'], styleWeights['weight2'])

In [29]:
f, ax = plt.subplots(2, 1)
ax[0].hist(styleWeights['weight1'], 40)
ax[1].hist(styleWeights['weight2'], 40)

(array([4.320e+04, 1.277e+03, 4.910e+02, 2.690e+02, 1.400e+02, 1.010e+02,
        7.900e+01, 5.300e+01, 5.200e+01, 2.800e+01, 2.000e+01, 1.500e+01,
        2.200e+01, 1.100e+01, 8.000e+00, 1.000e+01, 4.000e+00, 2.000e+00,
        3.000e+00, 0.000e+00, 4.000e+00, 2.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 2.000e+00]),
 array([8.54649722e-06, 7.99730342e-03, 1.59860603e-02, 2.39748173e-02,
        3.19635742e-02, 3.99523311e-02, 4.79410881e-02, 5.59298450e-02,
        6.39186019e-02, 7.19073588e-02, 7.98961158e-02, 8.78848727e-02,
        9.58736296e-02, 1.03862387e-01, 1.11851143e-01, 1.19839900e-01,
        1.27828657e-01, 1.35817414e-01, 1.43806171e-01, 1.51794928e-01,
        1.59783685e-01, 1.67772442e-01, 1.75761199e-01, 1.83749956e-01,
        1.91738713e-01, 1.99727470e-01, 2.07716227e-01, 2.15704983e-01

In [234]:
# prune graph by only allowing each node to connect to at most N other nodes, which will be the ones with the largest weights

# build graph manually using networkx
# allows us to apply algorithms like minimum spanning tree

Ggenre = nx.from_pandas_edgelist(genreWeights, source='g1', target='g2', edge_attr='weight')
# Ggenre = nx.from_pandas_edgelist(genreWeights, source='from', target='to', edge_attr='weight')

Ggenre_MinST = nx.minimum_spanning_tree(Ggenre)
Ggenre_MaxST = nx.maximum_spanning_tree(Ggenre)

genreWeightsMinST = nx.to_pandas_edgelist(Ggenre_MinST)
genreWeightsMaxST = nx.to_pandas_edgelist(Ggenre_MaxST)

# --

Gstyle = nx.from_pandas_edgelist(styleWeights, source='g1', target='g2', edge_attr='weight')
# Gstyle = nx.from_pandas_edgelist(styleWeights, source='from', target='to', edge_attr='weight')

Gstyle_MinST = nx.minimum_spanning_tree(Gstyle)
Gstyle_MaxST = nx.maximum_spanning_tree(Gstyle)

styleWeightsMinST = nx.to_pandas_edgelist(Gstyle_MinST)
styleWeightsMaxST = nx.to_pandas_edgelist(Gstyle_MaxST)

In [380]:
# degree centrality

degreeCentral = pd.Series(nx.degree_centrality(Gstyle_MaxST))
degreeCentral = degreeCentral / degreeCentral.max()
# degreeCentral.sort_values(ascending=False).head(10)

# closeness centrality

closeCentral = pd.Series(nx.closeness_centrality(Gstyle_MaxST))
closeCentral = closeCentral / closeCentral.max()
# closeCentral.sort_values(ascending=False).head(10)

# harmonic centrality

harmonicCentral = pd.Series(nx.harmonic_centrality(Gstyle_MaxST))
harmonicCentral = harmonicCentral[degreeCentral.index]
harmonicCentral = harmonicCentral / harmonicCentral.max()
# harmonicCentral.sort_values(ascending=False).head(10)

# betweenness centrality

btwnCentral = pd.Series(nx.betweenness_centrality(Gstyle_MaxST))
btwnCentral = btwnCentral / btwnCentral.max()
# btwnCentral.sort_values(ascending=False).head(10)

# eigenvector centrality

eigenCen = pd.Series(nx.eigenvector_centrality(Gstyle_MaxST,
                          max_iter=1000,
                          tol=1e-05,))
eigenCen = eigenCen / eigenCen.max()
# eigenCen.sort_values(ascending=False).head(10)

# eigenvector centrality - weighted

eigenCenWt = pd.Series(nx.eigenvector_centrality(Gstyle_MaxST,
                          max_iter=1000,
                          tol=1e-05,
                          weight='weight'))
eigenCenWt = eigenCenWt / eigenCenWt.max()
# eigenCenWt.sort_values(ascending=False).head(10)

# katz centrality

katzCentral = pd.Series(nx.katz_centrality(Gstyle_MaxST,
                                          normalized=True))
katzCentral = katzCentral / katzCentral.max()
# katzCentral.sort_values(ascending=False).head(10)

# katz centrality - weighted

katzCentralWt = pd.Series(nx.katz_centrality(Gstyle_MaxST,
                                            normalized=True))
katzCentralWt = katzCentralWt / katzCentralWt.max()
# katzCentralWt.sort_values(ascending=False).head(10)

# page rank

pageRank = pd.Series(nx.pagerank(Gstyle_MaxST))
pageRank = pageRank / pageRank.max()
# pageRank.sort_values(ascending=False).head(10)

In [382]:
centralityDF = pd.concat((degreeCentral, closeCentral, harmonicCentral, btwnCentral,
                          eigenCen, eigenCenWt, katzCentral, pageRank),
                        axis=1)
centralityDF.columns = ('degreeCentral', 'closeCentral', 'harmonicCentral',
                        'btwnCentral', 'eigenCen', 'eigenCenWt',
                        'katzCentral', 'pageRank', )

In [383]:
centralityDF['grandCentrality'] = centralityDF.mean(axis=1)

In [397]:
centralityDF['grandCentrality2'] = centralityDF[['harmonicCentral', 'btwnCentral', 'pageRank']].mean(axis=1)

In [388]:
centralityDF['grandCentrality'].sort_values(ascending=False).head(10)

Ambient             0.774380
Psychedelic Rock    0.723136
Experimental        0.687129
Folk Rock           0.687106
Prog Rock           0.638532
Indie Rock          0.613741
Downtempo           0.607540
Cha-Cha             0.592291
Hindustani          0.585768
Industrial          0.553193
Name: grandCentrality, dtype: float64

In [398]:
centralityDF['grandCentrality2'].sort_values(ascending=False).head(10)

Experimental        0.901491
Ambient             0.830740
Prog Rock           0.766390
Cha-Cha             0.734066
Psychedelic Rock    0.707612
Industrial          0.692701
Noise               0.676935
Fusion              0.654353
Avantgarde          0.634425
Art Rock            0.633896
Name: grandCentrality2, dtype: float64

In [392]:
centralityDF.columns.to_list()

['degreeCentral',
 'closeCentral',
 'harmonicCentral',
 'btwnCentral',
 'eigenCen',
 'eigenCenWt',
 'katzCentral',
 'pageRank',
 'grandCentrality']

In [393]:
# bad: degreeCentral, closeCentral, eigenCen, eigenCenWt
# good: harmonicCentral, btwnCentral, pageRank

for colName in centralityDF.columns.to_list():
    print(colName)
    print(centralityDF[colName].sort_values(ascending=False).head(10))

degreeCentral
Ambient             1.000000
Folk Rock           0.857143
Juke                0.857143
African             0.857143
Cha-Cha             0.857143
Psychedelic Rock    0.857143
Indie Rock          0.857143
Sephardic           0.857143
Breakbeat           0.714286
Ballad              0.714286
Name: degreeCentral, dtype: float64
closeCentral
Experimental        1.000000
Avantgarde          0.992607
Art Rock            0.985008
Prog Rock           0.977215
Noise               0.956333
Ambient             0.943477
Jazz-Rock           0.937462
Abstract            0.923998
Psychedelic Rock    0.920692
Industrial          0.915505
Name: closeCentral, dtype: float64
harmonicCentral
Ambient             1.000000
Experimental        0.998933
Psychedelic Rock    0.959933
Prog Rock           0.957563
Avantgarde          0.935197
Noise               0.932178
Downtempo           0.929322
Art Rock            0.922472
Industrial          0.903630
Modern Classical    0.891745
Name: harmonicCe

In [187]:
# build graph manually using pyvis (a bit more control)

netGenre = Network(notebook=True)

netGenre.add_nodes(list(range(genreSizes.shape[0])),
             value=genreSizes['size'].to_list(),
             label=genreSizes.index.to_list(),)

# build a dict that maps the node labels to their id number

label2id = {}
for node in netGenre.nodes:
    label2id[node['label']] = node['id']

edgeTups = list(genreWeightsSparse[['g1', 'g2']].to_records(index=False))
# weights = genreWeightsSparse['weight'].to_list()
weights = genreWeightsSparse['normWeight'].to_list()
for edgeTup, weight in zip(edgeTups, weights):
    netGenre.add_edge(label2id[edgeTup[0]],
                 label2id[edgeTup[1]],
                 value=weight)

# netGenre.toggle_physics(False)

netGenre.show('example.html')

In [34]:
f, ax = plt.subplots()
ax.hist(styleWeights['normWeight'][styleWeights['normWeight'] < .02], 20)

(array([15686.,  8568.,  4527.,  2923.,  1916.,  1409.,  1153.,   928.,
          706.,   625.,   529.,   471.,   393.,   313.,   297.,   265.,
          277.,   232.,   235.,   184.]),
 array([3.00002891e-05, 1.02831428e-03, 2.02662827e-03, 3.02494226e-03,
        4.02325625e-03, 5.02157024e-03, 6.01988423e-03, 7.01819823e-03,
        8.01651222e-03, 9.01482621e-03, 1.00131402e-02, 1.10114542e-02,
        1.20097682e-02, 1.30080822e-02, 1.40063962e-02, 1.50047102e-02,
        1.60030241e-02, 1.70013381e-02, 1.79996521e-02, 1.89979661e-02,
        1.99962801e-02]),
 <BarContainer object of 20 artists>)

In [34]:
# build graph manually (a bit more control)

netStyle = Network(notebook=True)

netStyle.add_nodes(list(range(styleSizes.shape[0])),
             value=styleSizes['cnt'].to_list(),
             label=styleSizes.index.to_list(),)

# build a dict that maps the node labels to their id number

label2id = {}
for node in netStyle.nodes:
    label2id[node['label']] = node['id']

edgeTups = list(styleWeightsMaxST[['source', 'target']].to_records(index=False))
weights = styleWeightsMaxST['weight'].to_list()
for edgeTup, weight in zip(edgeTups, weights):
    netStyle.add_edge(label2id[edgeTup[0]],
                 label2id[edgeTup[1]],
                 value=weight)

# netStyle.toggle_physics(False)

netStyle.show('example.html')

In [10]:


# the node_df needs id, size, and color
# the edge_df needs from, to, and weight

# from jaal.datasets import load_got
# edge_df, node_df = load_got()  # for an example

In [157]:
# genres - all

node_df = genreSizes.reset_index()
node_df['id'] = node_df['genre']
edge_df = genreWeights.rename(columns={'g1': 'from', 'g2': 'to', })

In [156]:
# styles - all

node_df = styleSizes.reset_index()
node_df['id'] = node_df['style']
edge_df = styleWeights.rename(columns={'g1': 'from', 'g2': 'to', })
edge_df = edge_df.loc[edge_df['normWeight'] > .05]

KeyError: 'normWeight'

In [218]:
# styles - MST

node_df = styleSizes.reset_index()
node_df['id'] = node_df['style']
edge_df = styleWeightsMaxST.rename(columns={'source': 'from', 'target': 'to', })

In [75]:
node_df.to_csv('styleNodes.csv')
edge_df.to_csv('styleEdges.csv')

In [159]:
# genres - MST

node_df = genreSizes.reset_index()
node_df['id'] = node_df['genre']
edge_df = genreWeightsMaxST.rename(columns={'source': 'from', 'target': 'to', })

In [219]:
Jaal(edge_df, node_df).plot()

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jul/2022 16:47:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2022 16:47:32] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


127.0.0.1 - - [06/Jul/2022 16:47:39] "POST /_dash-update-component HTTP/1.1" 200 -


inside color node genre


127.0.0.1 - - [06/Jul/2022 16:47:40] "POST /_dash-update-component HTTP/1.1" 200 -


Modifying node size using  cnt


127.0.0.1 - - [06/Jul/2022 16:47:41] "POST /_dash-update-component HTTP/1.1" 200 -


Modifying edge size using  weight


In [199]:
# Taken from https://stackoverflow.com/questions/470690/how-to-automatically-generate-n-distinct-colors
KELLY_COLORS_HEX = [
    "#FFB300", # Vivid Yellow
    "#803E75", # Strong Purple
    "#FF6800", # Vivid Orange
    "#A6BDD7", # Very Light Blue
    "#C10020", # Vivid Red
    "#CEA262", # Grayish Yellow
    "#817066", # Medium Gray

    # The following don't work well for people with defective color vision
    "#007D34", # Vivid Green
    "#F6768E", # Strong Purplish Pink
    "#00538A", # Strong Blue
    "#FF7A5C", # Strong Yellowish Pink
    "#53377A", # Strong Violet
    "#FF8E00", # Vivid Orange Yellow
    "#B32851", # Strong Purplish Red
    "#F4C800", # Vivid Greenish Yellow
    "#7F180D", # Strong Reddish Brown
    "#93AA00", # Vivid Yellowish Green
    "#593315", # Deep Yellowish Brown
    "#F13A13", # Vivid Reddish Orange
    "#232C16", # Dark Olive Green
    ]

In [227]:
# genres - MST

node_df_genre = genreSizes.reset_index().rename(columns={'genre': 'label', 'cnt': 'value'})
id2color = {i: KELLY_COLORS_HEX[i] for i in node_df_genre['id']}
node_df_genre['color'] = [id2color[i] for i in node_df_genre['id']]

edge_df_genre = genreWeightsMaxST.rename(columns={'source': 'from', 'target': 'to', 'weight': 'value'})

edge_df_genre['value'] = edge_df_genre['value'] / edge_df_genre['value'].max()

node_df_genre.to_json('genreNodes.json', orient='records')
edge_df_genre.to_json('genreEdges.json', orient='records')

# styles - MST

node_df_style = styleSizes.reset_index().rename(columns={'style': 'label', 'cnt': 'value'})
node_df_style['color'] = [id2color[label2id_genre[lbl]] for lbl in node_df_style['genre']]
node_df_style.drop(columns=('genre'), inplace=True)

edge_df_style = styleWeightsMaxST.rename(columns={'source': 'from', 'target': 'to', 'weight': 'value'})
edge_df_style['value'] = edge_df_style['value'] / edge_df_style['value'].max()

node_df_style.to_json('styleNodes.json', orient='records')
edge_df_style.to_json('styleEdges.json', orient='records')